In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-airline-sentiment/Tweets.csv
/kaggle/input/twitter-airline-sentiment/database.sqlite


In [4]:
tweet_data = pd.read_csv('/kaggle/input/twitter-airline-sentiment/Tweets.csv')
print("data shape:",tweet_data.shape)
print("what are columns:",tweet_data.columns)
tweet_data.head()

data shape: (14640, 15)
what are columns: Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
tweet_data = tweet_data.drop(['tweet_id','retweet_count', 'tweet_coord', 'tweet_created',
                               'tweet_location','name','user_timezone'],axis = 1)

In [6]:
tweet_data.head()


,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,negativereason_gold,text
0,neutral,1.0000,NaN,NaN,Virgin America,NaN,NaN,@VirginAmerica What @dhepburn said.
1,positive,0.3486,NaN,0.0000,Virgin America,NaN,NaN,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,NaN,NaN,Virgin America,NaN,NaN,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,NaN,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,NaN,@VirginAmerica and it's a really big bad thing...


In [7]:
tweet_data['negativereason_gold'].unique()


array([nan, 'Late Flight\nFlight Attendant Complaints',
       'Late Flight\nLost Luggage', 'Late Flight',
       'Customer Service Issue', "Can't Tell",
       'Cancelled Flight\nCustomer Service Issue', 'Cancelled Flight',
       'Bad Flight', 'Lost Luggage\nDamaged Luggage',
       'Late Flight\nCancelled Flight', 'Flight Attendant Complaints',
       'Customer Service Issue\nLost Luggage',
       "Customer Service Issue\nCan't Tell"], dtype=object)

In [8]:
tweet_data['negativereason'] = tweet_data['negativereason'].fillna('')
tweet_data['negativereason_confidence'] = tweet_data['negativereason_confidence'].fillna(0)
tweet_data['airline_sentiment_gold'] = tweet_data['airline_sentiment_gold'].fillna('')
tweet_data['negativereason_gold'] = tweet_data['negativereason_gold'].fillna('')

In [9]:
tweet_data.head()


,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,negativereason_gold,text
0,neutral,1.0000,,0.0000,Virgin America,,,@VirginAmerica What @dhepburn said.
1,positive,0.3486,,0.0000,Virgin America,,,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,,0.0000,Virgin America,,,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,,,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,,,@VirginAmerica and it's a really big bad thing...


In [10]:
print("different topics of negative reasons are:",tweet_data['negativereason'].unique())

different topics of negative reasons are: ['' 'Bad Flight' "Can't Tell" 'Late Flight' 'Customer Service Issue'
 'Flight Booking Problems' 'Lost Luggage' 'Flight Attendant Complaints'
 'Cancelled Flight' 'Damaged Luggage' 'longlines']


In [11]:
!pip install contractions


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.8 MB/s eta 0:00:00


In [12]:
from nltk.corpus import stopwords
import string
import re
import contractions
def text_cleaning(text):
    #not removing the stopwords so that the sentences stay normal.
    #forbidden_words = set(stopwords.words('english'))
    if text:
        text = contractions.fix(text)
        text = ' '.join(text.split('.'))
        text = re.sub(r'\s+', ' ', re.sub('[^A-Za-z0-9]', ' ', text.strip().lower())).strip()
        text = re.sub(r'\W+', ' ', text.strip().lower()).strip()
        text = [word for word in text.split()]
        return text
    return []

In [13]:
tweet_data['text'] = tweet_data['text'].apply(lambda x: ' '.join(text_cleaning(x)))

In [14]:
tweet_data.head(20)

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,negativereason_gold,text
0,neutral,1.0000,,0.0000,Virgin America,,,virginamerica what dhepburn said
1,positive,0.3486,,0.0000,Virgin America,,,virginamerica plus you have added commercials ...
2,neutral,0.6837,,0.0000,Virgin America,,,virginamerica i did not today must mean i need...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,,,virginamerica it is really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,,,virginamerica and it is a really big bad thing...
5,negative,1.0000,Can't Tell,0.6842,Virgin America,,,virginamerica seriously would pay 30 a flight ...
6,positive,0.6745,,0.0000,Virgin America,,,virginamerica yes nearly every time i fly vx t...
7,neutral,0.6340,,0.0000,Virgin America,,,virginamerica really missed a prime opportunit...
8,positive,0.6559,,0.0000,Virgin America,,,virginamerica well i did not but now i do d
9,positive,1.0000,,0.0000,Virgin America,,,virginamerica it was amazing and arrived an ho...


***Sentiment analysis Using Nltk:***

In [15]:
import nltk 

nltk.download(["names","stopwords","state_union","twitter_samples",
              "movie_reviews","averaged_perceptron_tagger","vader_lexicon",
              "punkt"])

[nltk_data] Downloading package names to /usr/share/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to /usr/share/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-

True

Using NLTK's pretrained Senitment anlyser Vader:


In [16]:
from nltk.sentiment import SentimentIntensityAnalyzer as SIA
sia = SIA()
print(sia.polarity_scores('hey this carrot is not great'))

{'neg': 0.397, 'neu': 0.603, 'pos': 0.0, 'compound': -0.5096}


/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [17]:
texts = tweet_data['text'].tolist()
negative_scores = []
neutral_scores = []
positive_scores = []
compound_scores = []
final_tag = []
for text in texts:
    score_dictionary = sia.polarity_scores(text)
    negative_scores.append(score_dictionary['neg'])
    positive_scores.append(score_dictionary['pos'])
    neutral_scores.append(score_dictionary['neu'])
    compound_scores.append(score_dictionary['compound'])
    if score_dictionary['compound']>0:
        final_tag.append('positive')
    elif score_dictionary['compound']<0:
        final_tag.append('negative')
    else:
        final_tag.append('neutral')
tweet_data['negative_score'] = negative_scores
tweet_data['positive_score'] = positive_scores
tweet_data['neutral_score'] = neutral_scores
tweet_data['compound_score'] = compound_scores
tweet_data['final_tag'] = final_tag

In [18]:
tweet_data.head(20)


,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,negativereason_gold,text,negative_score,positive_score,neutral_score,compound_score,final_tag
0,neutral,1.0000,,0.0000,Virgin America,,,virginamerica what dhepburn said,0.000,0.000,1.000,0.0000,neutral
1,positive,0.3486,,0.0000,Virgin America,,,virginamerica plus you have added commercials ...,0.000,0.000,1.000,0.0000,neutral
2,neutral,0.6837,,0.0000,Virgin America,,,virginamerica i did not today must mean i need...,0.000,0.000,1.000,0.0000,neutral
3,negative,1.0000,Bad Flight,0.7033,Virgin America,,,virginamerica it is really aggressive to blast...,0.216,0.123,0.661,-0.2716,negative
4,negative,1.0000,Can't Tell,1.0000,Virgin America,,,virginamerica and it is a really big bad thing...,0.296,0.000,0.704,-0.5829,negative
5,negative,1.0000,Can't Tell,0.6842,Virgin America,,,virginamerica seriously would pay 30 a flight ...,0.238,0.069,0.693,-0.5945,negative
6,positive,0.6745,,0.0000,Virgin America,,,virginamerica yes nearly every time i fly vx t...,0.000,0.172,0.828,0.4019,positive
7,neutral,0.6340,,0.0000,Virgin America,,,virginamerica really missed a prime opportunit...,0.142,0.175,0.683,0.1458,positive
8,positive,0.6559,,0.0000,Virgin America,,,virginamerica well i did not but now i do d,0.000,0.205,0.795,0.1406,positive
9,positive,1.0000,,0.0000,Virgin America,,,virginamerica it was amazing and arrived an ho...,0.000,0.340,0.660,0.7717,positive


In [19]:
texts[17]

'virginamerica i flew from nyc to sfo last week and could not fully sit in my seat due to two large gentleman on either side of me help'

In [20]:
from sklearn.metrics import classification_report as crep
print("sentiment analysis performance for nltk:")
print(crep(tweet_data['airline_sentiment'],tweet_data['final_tag']))

sentiment analysis performance for nltk:
              precision    recall  f1-score   support

    negative       0.90      0.45      0.60      9178
     neutral       0.43      0.32      0.37      3099
    positive       0.27      0.90      0.42      2363

    accuracy                           0.49     14640
   macro avg       0.53      0.56      0.46     14640
weighted avg       0.70      0.49      0.52     14640



Using TextBlob: 

In [21]:
!pip install -U textblob
!python -m textblob.download_corpora

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
[nltk_data] Downloading package brown to /usr/share/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is

In [22]:
from textblob import TextBlob

tb1 = TextBlob('I just am trying textblob first time.')
tb1

TextBlob("I just am trying textblob first time.")

In [23]:
tb1.words

WordList(['I', 'just', 'am', 'trying', 'textblob', 'first', 'time'])

In [24]:
tb1.sentences

[Sentence("I just am trying textblob first time.")]

In [25]:
tb1.noun_phrases

WordList([])

In [26]:
tb3=TextBlob(" We are learning cool Library . We are enjoying a lot .")
tb3.noun_phrases

WordList(['cool library'])

In [27]:
tb2=TextBlob("Tags will give the POS for all the words.")
tb2.tags

[('Tags', 'NNP'),
 ('will', 'MD'),
 ('give', 'VB'),
 ('the', 'DT'),
 ('POS', 'NNP'),
 ('for', 'IN'),
 ('all', 'PDT'),
 ('the', 'DT'),
 ('words', 'NNS')]

In [28]:
doc2=TextBlob("i hate dawgs")
doc2.polarity

-0.8

In [29]:
texts = tweet_data['text'].tolist()
textblob_score = []
textblob_tag = []
for text in texts:
    doc_current = TextBlob(text)
    score = doc_current.polarity
    textblob_score.append(score)
    if score > 0:
        textblob_tag.append('positive')
    elif score<0:
        textblob_tag.append('negative')
    else:
        textblob_tag.append('neutral')
tweet_data['textblob_score'] = textblob_score
tweet_data['textblob_sentiment_tag'] = textblob_tag

In [30]:
tweet_data[['airline_sentiment','text','textblob_score','textblob_sentiment_tag']].head(20)

,airline_sentiment,text,textblob_score,textblob_sentiment_tag
0,neutral,virginamerica what dhepburn said,0.000000,neutral
1,positive,virginamerica plus you have added commercials ...,0.000000,neutral
2,neutral,virginamerica i did not today must mean i need...,-0.312500,negative
3,negative,virginamerica it is really aggressive to blast...,0.006250,positive
4,negative,virginamerica and it is a really big bad thing...,-0.350000,negative
5,negative,virginamerica seriously would pay 30 a flight ...,-0.208333,negative
6,positive,virginamerica yes nearly every time i fly vx t...,0.450000,positive
7,neutral,virginamerica really missed a prime opportunit...,0.200000,positive
8,positive,virginamerica well i did not but now i do d,0.000000,neutral
9,positive,virginamerica it was amazing and arrived an ho...,0.466667,positive


In [31]:
from sklearn.metrics import classification_report as crep
print("sentiment analysis performance for textblob:")
print(crep(tweet_data['airline_sentiment'],tweet_data['textblob_sentiment_tag']))

sentiment analysis performance for textblob:
              precision    recall  f1-score   support

    negative       0.88      0.35      0.50      9178
     neutral       0.32      0.56      0.41      3099
    positive       0.32      0.76      0.45      2363

    accuracy                           0.46     14640
   macro avg       0.51      0.56      0.46     14640
weighted avg       0.67      0.46      0.47     14640



**Huggingface Transformer based pre trained models for sentiment analysis***

Bert Based transformer:

In [40]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')
classifier("I am so happy to use huggingface today!")[0]['label']

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


'POSITIVE'

In [50]:
classifier("") 

[{'label': 'NEGATIVE', 'score': 0.9879854321479797}]

In [52]:
tweet_data['bert_based_sentiment'] = tweet_data['text'].apply(lambda x: classifier(x)[0]['label'].lower())
tweet_data_cut = tweet_data[tweet_data['airline_sentiment']!='neutral']
print(crep(tweet_data_cut['airline_sentiment'],tweet_data_cut['bert_based_sentiment']))

              precision    recall  f1-score   support

    negative       0.95      0.91      0.93      9178
    positive       0.69      0.81      0.75      2363

    accuracy                           0.89     11541
   macro avg       0.82      0.86      0.84     11541
weighted avg       0.90      0.89      0.89     11541



Bart model:

In [53]:
classifier_febu = pipeline('sentiment-analysis',model = 'facebook/bart-large')
classifier_febu("I hate it!")
classifier_febu("I love you!")

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[{'label': 'LABEL_1', 'score': 0.4281269311904907}]

In [54]:
def sentiment_analysis(text):
    sentiment = classifier_febu(text)[0]['label']
    if sentiment == 'LABEL_0': return 'positive'
    return 'negative'

In [ ]:
tweet_data['facebook_bert_based_sentiment'] = tweet_data['text'].apply(sentiment_analysis)
tweet_data_cut = tweet_data[tweet_data['airline_sentiment']!='neutral']
print(crep(tweet_data_cut['airline_sentiment'],tweet_data_cut['facebook_bert_based_sentiment']))

In [ ]:
classifier_third = pipeline('sentiment-analysis',model = 'cardiffnlp/twitter-roberta-base-sentiment')
print(classifier_third("I hate it!"))
print(classifier_third("I love you!"))
print(classifier_third("I don't know about the routine"))
def sentiment_analysis_third(text):
    sentiment = classifier_third(text)[0]['label']
    if sentiment == 'LABEL_0': return 'negative'
    elif sentiment == 'LABEL_1': return 'neutral'
    elif sentiment == 'LABEL_2': return 'positive'
    return 'neutral'
tweet_data['facebook_bert_based_sentiment'] = tweet_data['text'].apply(sentiment_analysis_third)
print(crep(tweet_data['airline_sentiment'],tweet_data['facebook_bert_based_sentiment']))

In [ ]:
!pip install flair

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')
sentence = Sentence('Flair is pretty neat!')
classifier.predict(sentence)
# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

In [ ]:
def sentiment_analysis_flair(text):
    sentence = Sentence(text)
    classifier.predict(sentence)
    return str(sentence.labels[0]).split("(")[0].lower()[:-1]

In [ ]:
tweet_data['flair_based_sentiment'] = tweet_data['text'].apply(sentiment_analysis_flair)
tweet_data_cut = tweet_data[tweet_data['airline_sentiment']!='neutral']
print(crep(tweet_data_cut['airline_sentiment'],tweet_data_cut['flair_based_sentiment']))